In [84]:
import re
import os 
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import sparse
import ast

import seaborn as sns
import matplotlib.pyplot  as plt

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

import keras
from keras.layers import Input, Dense

import gensim 
from gensim.models import Word2Vec 

from sklearn import model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

import pickle

import textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

In [80]:
class DataLake:
    
    path = 'features/'
    
    def __init__(self, version):
        self.version = version 
        self.version_path = self.path + version + "/"
        
        #create version folder if not exists
        if not os.path.exists(self.path + self.version):
            try:
                os.makedirs(self.path + self.version)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise
    
    def save_obj(self, obj, file_name):
        with open(self.version_path + file_name, 'wb') as output:  # Overwrites any existing file.
            pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
    def save_npz(self, obj, file_name):
        sparse.save_npz(self.version_path + file_name, obj)
    
    def load_npz(self, file_name):
        return sparse.load_npz(self.version_path + file_name)
    
    def load_obj(self, obj_name):
        with open(self.version_path + obj_name, 'rb') as input:
            return pickle.load(input)

    def load_config(self, file_name):
        config_path = 'configs/' + self.version + "/"
        file = open(config_path + file_name, "r")
        contents = file.read()
        return ast.literal_eval(contents)
    
data_lake = DataLake(version='v3')

In [72]:
def prepare_data(df_to_prepare, key='text_normalized'):
    train_x = df_to_prepare[key].tolist()
    train_y = df_to_prepare.category.tolist()
    return (train_x, train_y)

In [73]:
def load_feature(feature, load_version=None):

    if load_version is None:
        data_lake_ = data_lake
    else:
        data_lake_ = DataLake(version=load_version)
    
    df[feature] = data_lake_.load_obj(feature + '.pkl')

# Loading features 

In [74]:
df = data_lake.load_obj('df-cleaned.pkl')

f_name = 'text_normalized'
load_feature(f_name)

df_train_table = df[df.path == 'dataset/train_set/']
df_test_table = df[df.path == 'dataset/test_set/']

In [103]:
train_x, train_y = prepare_data(df_train_table)
valid_x, valid_y = prepare_data(df_test_table)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


In [ ]:
#removing feature that we wont't use anymore
del df[f_name]

## RNN & LDA topics

In [99]:
def create_rnn(input_shape_value, n_hidden_layers):
    model = keras.Sequential()
    model = keras.Sequential([keras.layers.Dense(64, activation='relu', input_shape=[input_shape_value])])
    
    for i in range(n_hidden_layers):
        model.add(keras.layers.Dense(128, activation='relu'))

    model.add(keras.layers.Dense(1))

    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [100]:
# load LDA structures
####################
X_topics = data_lake.load_obj('X_topics' + '.pkl')
lda_model = data_lake.load_obj('lda_model' + '.pkl')#lda model was trained with tfidf_vect_ngram

# ngram level tf-idf 
####################
xvalid_tfidf_ngram = data_lake.load_npz('xvalid_tfidf_ngram' + '.npz')
xtrain_tfidf_ngram = data_lake.load_npz('xtrain_tfidf_ngram' + '.npz')

In [101]:
lda_train_data = lda_model.transform(xtrain_tfidf_ngram)
lda_valid_data = lda_model.transform(xvalid_tfidf_ngram)

In [104]:
# xvalid_tfidf_ngram Y xtrain_tfidf_ngram ESTAN CREADOS CON EL DATASET QUE TIENE EL ACAK !!!
#

try:
    model = data_lake.load_obj('rnn_model.pkl')

except:
    print("fitting RNN...")
    time_start = time.time()
    
    model = create_rnn(input_shape_value=len(lda_model.components_), n_hidden_layers=4)
    EPOCHS = 1000

    model.fit(
      lda_train_data, train_y,
      epochs=EPOCHS, batch_size=128, validation_split = 0.2, verbose=1)

    data_lake.save_obj(model, 'rnn_model.pkl')
    
    print("fitting finished - time: ", time.time() - time_start)
    print()

fitting RNN...
Train on 41887 samples, validate on 10472 samples
Epoch 1/1000
41887/41887 [==============================] - 1s 29us/step - loss: 0.9309 - val_loss: 6.7589
Epoch 2/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.4205 - val_loss: 7.4177
Epoch 3/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.3544 - val_loss: 6.0132
Epoch 4/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.3196 - val_loss: 7.1679
Epoch 5/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.2943 - val_loss: 6.3987
Epoch 6/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.2778 - val_loss: 6.4326
Epoch 7/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.2651 - val_loss: 6.1301
Epoch 8/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.2531 - val_loss: 5.6558
Epoch 9/1000
41887/41887 [==============================] - 1s 17us/step - loss

41887/41887 [==============================] - 1s 18us/step - loss: 0.1301 - val_loss: 6.0762
Epoch 77/1000
41887/41887 [==============================] - 1s 18us/step - loss: 0.1308 - val_loss: 6.1108
Epoch 78/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.1301 - val_loss: 5.9239
Epoch 79/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.1312 - val_loss: 5.9525
Epoch 80/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.1259 - val_loss: 5.9365
Epoch 81/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.1290 - val_loss: 5.9764
Epoch 82/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.1248 - val_loss: 5.7257
Epoch 83/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.1271 - val_loss: 5.8512
Epoch 84/1000
41887/41887 [==============================] - 1s 21us/step - loss: 0.1297 - val_loss: 6.0913
Epoch 85/1000
41887/41887 [===============

41887/41887 [==============================] - 1s 19us/step - loss: 0.0932 - val_loss: 5.5224
Epoch 152/1000
41887/41887 [==============================] - 1s 22us/step - loss: 0.0944 - val_loss: 5.7833
Epoch 153/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.1000 - val_loss: 5.3808
Epoch 154/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0935 - val_loss: 5.7896
Epoch 155/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.1002 - val_loss: 5.7403
Epoch 156/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0956 - val_loss: 5.4909
Epoch 157/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0908 - val_loss: 5.3497
Epoch 158/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0951 - val_loss: 5.7223
Epoch 159/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0938 - val_loss: 5.5341
Epoch 160/1000
41887/41887 [======

41887/41887 [==============================] - 1s 15us/step - loss: 0.0776 - val_loss: 5.7560
Epoch 227/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0730 - val_loss: 6.1254
Epoch 228/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0728 - val_loss: 5.6635
Epoch 229/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0751 - val_loss: 5.6675
Epoch 230/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0748 - val_loss: 5.4885
Epoch 231/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0759 - val_loss: 5.6543
Epoch 232/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0716 - val_loss: 5.5997
Epoch 233/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0709 - val_loss: 5.8430
Epoch 234/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0763 - val_loss: 5.6710
Epoch 235/1000
41887/41887 [======

41887/41887 [==============================] - 1s 21us/step - loss: 0.0616 - val_loss: 6.0647
Epoch 302/1000
41887/41887 [==============================] - 1s 21us/step - loss: 0.0660 - val_loss: 5.9719
Epoch 303/1000
41887/41887 [==============================] - 1s 22us/step - loss: 0.0642 - val_loss: 6.1055
Epoch 304/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0660 - val_loss: 6.1116
Epoch 305/1000
41887/41887 [==============================] - 1s 21us/step - loss: 0.0657 - val_loss: 5.7338
Epoch 306/1000
41887/41887 [==============================] - 1s 21us/step - loss: 0.0613 - val_loss: 5.9749
Epoch 307/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0621 - val_loss: 5.8548
Epoch 308/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0673 - val_loss: 5.9716
Epoch 309/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0616 - val_loss: 5.8929
Epoch 310/1000
41887/41887 [======

...

41887/41887 [==============================] - 1s 16us/step - loss: 0.0433 - val_loss: 5.9349
Epoch 677/1000
41887/41887 [==============================] - 1s 18us/step - loss: 0.0453 - val_loss: 6.3434
Epoch 678/1000
41887/41887 [==============================] - 1s 18us/step - loss: 0.0454 - val_loss: 6.0867
Epoch 679/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0432 - val_loss: 6.2270
Epoch 680/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0467 - val_loss: 6.0558
Epoch 681/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0424 - val_loss: 6.3569
Epoch 682/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0433 - val_loss: 6.6679
Epoch 683/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0468 - val_loss: 6.6136
Epoch 684/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0460 - val_loss: 6.1714
Epoch 685/1000
41887/41887 [======

41887/41887 [==============================] - 1s 15us/step - loss: 0.0482 - val_loss: 5.9821
Epoch 752/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0468 - val_loss: 6.1511
Epoch 753/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0472 - val_loss: 6.1954
Epoch 754/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0518 - val_loss: 6.1253
Epoch 755/1000
41887/41887 [==============================] - 1s 14us/step - loss: 0.0473 - val_loss: 5.9567
Epoch 756/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0474 - val_loss: 6.0798
Epoch 757/1000
41887/41887 [==============================] - 1s 24us/step - loss: 0.0470 - val_loss: 6.0175
Epoch 758/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0458 - val_loss: 5.8791
Epoch 759/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0509 - val_loss: 6.1612
Epoch 760/1000
41887/41887 [======

41887/41887 [==============================] - 1s 16us/step - loss: 0.0612 - val_loss: 6.7459
Epoch 827/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0617 - val_loss: 7.0028
Epoch 828/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0567 - val_loss: 5.7949
Epoch 829/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0604 - val_loss: 7.2344
Epoch 830/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0576 - val_loss: 7.0656
Epoch 831/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0602 - val_loss: 6.8477
Epoch 832/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0597 - val_loss: 7.1768
Epoch 833/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0588 - val_loss: 7.6595
Epoch 834/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0606 - val_loss: 7.9318
Epoch 835/1000
41887/41887 [======

41887/41887 [==============================] - 1s 26us/step - loss: 0.0491 - val_loss: 6.4265
Epoch 902/1000
41887/41887 [==============================] - 1s 18us/step - loss: 0.0445 - val_loss: 9.1389
Epoch 903/1000
41887/41887 [==============================] - 1s 26us/step - loss: 0.0511 - val_loss: 6.9682
Epoch 904/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0464 - val_loss: 7.3531
Epoch 905/1000
41887/41887 [==============================] - 1s 19us/step - loss: 0.0480 - val_loss: 7.3676
Epoch 906/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0487 - val_loss: 5.9324
Epoch 907/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0453 - val_loss: 6.2892
Epoch 908/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0457 - val_loss: 7.5999
Epoch 909/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0454 - val_loss: 7.4250
Epoch 910/1000
41887/41887 [======

41887/41887 [==============================] - 1s 17us/step - loss: 0.0460 - val_loss: 6.3712
Epoch 977/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0497 - val_loss: 5.7917
Epoch 978/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0450 - val_loss: 5.6312
Epoch 979/1000
41887/41887 [==============================] - 1s 17us/step - loss: 0.0470 - val_loss: 6.0232
Epoch 980/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0450 - val_loss: 6.3738
Epoch 981/1000
41887/41887 [==============================] - 1s 16us/step - loss: 0.0512 - val_loss: 5.6182
Epoch 982/1000
41887/41887 [==============================] - 1s 15us/step - loss: 0.0450 - val_loss: 5.5415
Epoch 983/1000
41887/41887 [==============================] - 1s 20us/step - loss: 0.0437 - val_loss: 5.5784
Epoch 984/1000
41887/41887 [==============================] - 1s 22us/step - loss: 0.0452 - val_loss: 5.9746
Epoch 985/1000
41887/41887 [======

In [107]:
letter_types = sorted(df.category.unique().tolist())

score = model.predict(lda_valid_data)
predictions = list(round(w) for [w] in score)

recall_info = ""

items_recall = recall_score(valid_y, predictions, average=None)
filtered_items_recall = zip(letter_types,items_recall)

for item in filtered_items_recall:
    print(str(item))

('AK', 0.4782608695652174)
('AR', 0.925)
('CL', 0.9871382636655949)
('ER', 0.45454545454545453)
('FL', 1.0)
('ND', 0.0)
('RL', 0.9738372093023255)
('TM', 0.25)
('UR', 1.0)
('UU', 0.0)
